<a href="https://colab.research.google.com/github/RyanS14/cafe-da-tia-rosa/blob/main/Coffee_Tia_rosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Sistema de Gestão - Coffee Shops Tia Rosa
Arquivo: sistema_coffee_tia_rosa.py
Descrição:
    Aplicação em linha de comando (CLI) escrita em Python que atende às necessidades do
    Coffee Shops Tia Rosa: gerenciamento de cardápio e produtos, controle de estoque,
    cadastro de clientes e programa de fidelidade, registro de pedidos, relatórios diários
    e persistência via SQLite.

Características principais:
    - Banco de dados SQLite (arquivo: tia_rosa.db)
    - CRUD para produtos
    - Exibição de cardápio amigável ao cliente
    - Registro de pedidos com atualização automática de estoque
    - Cadastro de clientes com programa de pontos (1 ponto por R$1 gasto, configurável)
    - Relatórios: vendas do dia, itens mais vendidos, cliente mais fiel
    - Importação/Exportação simples em CSV
    - Interface em português, projetada para usuários com pouca familiaridade tecnológica

Como usar:
    1) Execute: python sistema_coffee_tia_rosa.py
    2) Siga o menu principal. A maioria das operações possui mensagens explicativas.

Requisitos:
    - Python 3.8+
    - Módulos padrão: sqlite3, csv, datetime, tabulate (opcional — se não existir, o app usa um print simples)

Notas sobre segurança e melhorias futuras:
    - Para uso real em loja, considerar autenticação, backups automáticos e interface gráfica (web ou touchscreen).

Criado para fins educacionais / trabalho de disciplina.
"""

import sqlite3
import os
import csv
from datetime import datetime, date
from decimal import Decimal, ROUND_HALF_UP

# Tenta importar tabulate para tabelas legíveis; caso falte, prossegue sem ele
try:
    from tabulate import tabulate
    _HAS_TABULATE = True
except Exception:
    _HAS_TABULATE = False

DB_FILENAME = 'tia_rosa.db'
POINTS_PER_REAL = 1  # 1 ponto por R$1 gasto
CURRENCY = 'R$'

# -------------------------- Helpers --------------------------

def money(value):
    """Formata número como valor monetário brasileiro."""
    v = Decimal(value).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)
    return f"{CURRENCY} {v:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')


def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')


# -------------------------- Banco de dados --------------------------

class Database:
    def __init__(self, filename=DB_FILENAME):
        self.conn = sqlite3.connect(filename)
        self.conn.row_factory = sqlite3.Row
        self._ensure_schema()

    def _ensure_schema(self):
        c = self.conn.cursor()
        # Produtos
        c.execute('''
            CREATE TABLE IF NOT EXISTS products (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                description TEXT,
                price REAL NOT NULL,
                stock INTEGER NOT NULL DEFAULT 0,
                active INTEGER NOT NULL DEFAULT 1
            )
        ''')
        # Clientes
        c.execute('''
            CREATE TABLE IF NOT EXISTS customers (
                id INTEGER PRIMARY KEY,
                name TEXT NOT NULL,
                phone TEXT,
                email TEXT,
                points INTEGER NOT NULL DEFAULT 0
            )
        ''')
        # Pedidos e itens
        c.execute('''
            CREATE TABLE IF NOT EXISTS orders (
                id INTEGER PRIMARY KEY,
                customer_id INTEGER,
                created_at TEXT NOT NULL,
                total REAL NOT NULL,
                payment_method TEXT,
                FOREIGN KEY(customer_id) REFERENCES customers(id)
            )
        ''')
        c.execute('''
            CREATE TABLE IF NOT EXISTS order_items (
                id INTEGER PRIMARY KEY,
                order_id INTEGER NOT NULL,
                product_id INTEGER NOT NULL,
                qty INTEGER NOT NULL,
                price REAL NOT NULL,
                FOREIGN KEY(order_id) REFERENCES orders(id),
                FOREIGN KEY(product_id) REFERENCES products(id)
            )
        ''')
        self.conn.commit()

    def execute(self, query, params=()):
        c = self.conn.cursor()
        c.execute(query, params)
        self.conn.commit()
        return c

    def query(self, query, params=()):
        c = self.conn.cursor()
        c.execute(query, params)
        return c.fetchall()


# -------------------------- Regras de negócio --------------------------

class CoffeeSystem:
    def __init__(self, db: Database):
        self.db = db

    # ---------- Produtos ----------
    def add_product(self, name, description, price, stock):
        self.db.execute(
            'INSERT INTO products (name, description, price, stock) VALUES (?, ?, ?, ?)',
            (name.strip(), description.strip(), float(price), int(stock))
        )

    def list_products(self, only_active=True):
        if only_active:
            rows = self.db.query('SELECT * FROM products WHERE active=1 ORDER BY name')
        else:
            rows = self.db.query('SELECT * FROM products ORDER BY name')
        return rows

    def get_product(self, product_id):
        rows = self.db.query('SELECT * FROM products WHERE id=?', (product_id,))
        return rows[0] if rows else None

    def update_product(self, product_id, **fields):
        allowed = {'name', 'description', 'price', 'stock', 'active'}
        set_clauses = []
        params = []
        for k, v in fields.items():
            if k in allowed:
                set_clauses.append(f"{k}=?")
                params.append(v)
        if not set_clauses:
            return
        params.append(product_id)
        sql = f"UPDATE products SET {', '.join(set_clauses)} WHERE id=?"
        self.db.execute(sql, tuple(params))

    def delete_product(self, product_id):
        # Soft delete: marca como inativo
        self.update_product(product_id, active=0)

    # ---------- Clientes ----------
    def add_customer(self, name, phone=None, email=None):
        self.db.execute('INSERT INTO customers (name, phone, email) VALUES (?, ?, ?)',
                        (name.strip(), phone, email))

    def list_customers(self):
        return self.db.query('SELECT * FROM customers ORDER BY name')

    def find_customer_by_phone(self, phone):
        rows = self.db.query('SELECT * FROM customers WHERE phone=?', (phone,))
        return rows[0] if rows else None

    def get_customer(self, customer_id):
        rows = self.db.query('SELECT * FROM customers WHERE id=?', (customer_id,))
        return rows[0] if rows else None

    def add_points(self, customer_id, points):
        self.db.execute('UPDATE customers SET points = points + ? WHERE id=?', (int(points), customer_id))

    def redeem_points(self, customer_id, points):
        # Verifica saldo
        cust = self.get_customer(customer_id)
        if not cust:
            return False, 'Cliente não encontrado.'
        if cust['points'] < points:
            return False, 'Pontos insuficientes.'
        self.db.execute('UPDATE customers SET points = points - ? WHERE id=?', (points, customer_id))
        return True, 'Pontos resgatados.'

    # ---------- Pedidos ----------
    def create_order(self, customer_id, items, payment_method='Dinheiro'):
        # items: list of dicts {product_id, qty}
        total = 0
        for it in items:
            prod = self.get_product(it['product_id'])
            if not prod:
                raise ValueError(f"Produto {it['product_id']} não encontrado")
            if prod['stock'] < it['qty']:
                raise ValueError(f"Estoque insuficiente: {prod['name']}")
            total += prod['price'] * it['qty']

        now = datetime.now().isoformat()
        cur = self.db.execute('INSERT INTO orders (customer_id, created_at, total, payment_method) VALUES (?, ?, ?, ?)',
                              (customer_id, now, float(total), payment_method))
        order_id = cur.lastrowid

        for it in items:
            prod = self.get_product(it['product_id'])
            self.db.execute('INSERT INTO order_items (order_id, product_id, qty, price) VALUES (?, ?, ?, ?)',
                            (order_id, it['product_id'], it['qty'], prod['price']))
            # decrementa estoque
            new_stock = prod['stock'] - it['qty']
            self.update_product(it['product_id'], stock=new_stock)

        # Adiciona pontos ao cliente
        if customer_id:
            earned = int(total) * POINTS_PER_REAL
            self.add_points(customer_id, earned)

        return order_id, total

    # ---------- Relatórios ----------
    def sales_on_date(self, target_date: date):
        start = datetime.combine(target_date, datetime.min.time()).isoformat()
        end = datetime.combine(target_date, datetime.max.time()).isoformat()
        rows = self.db.query('SELECT * FROM orders WHERE created_at BETWEEN ? AND ?', (start, end))
        total = sum(r['total'] for r in rows)
        return rows, total

    def top_sold_items(self, limit=5):
        rows = self.db.query('''
            SELECT p.id, p.name, SUM(oi.qty) as sold
            FROM order_items oi
            JOIN products p ON p.id = oi.product_id
            GROUP BY p.id, p.name
            ORDER BY sold DESC
            LIMIT ?
        ''', (limit,))
        return rows

    def customer_rank(self, limit=5):
        rows = self.db.query('SELECT id, name, points FROM customers ORDER BY points DESC LIMIT ?', (limit,))
        return rows

    # ---------- Import/Export ----------
    def import_products_csv(self, path):
        imported = 0
        with open(path, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                try:
                    name = row.get('name') or row.get('nome')
                    description = row.get('description') or row.get('descricao') or ''
                    price = float(row.get('price') or row.get('preco') or 0)
                    stock = int(row.get('stock') or row.get('estoque') or 0)
                    self.add_product(name, description, price, stock)
                    imported += 1
                except Exception:
                    continue
        return imported

    def export_products_csv(self, path):
        rows = self.list_products(only_active=False)
        with open(path, 'w', newline='', encoding='utf-8') as f:
            fieldnames = ['id', 'name', 'description', 'price', 'stock', 'active']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for r in rows:
                writer.writerow({k: r[k] for k in fieldnames})
        return len(rows)


# -------------------------- Interface CLI --------------------------

SYSTEM = CoffeeSystem(Database())


def print_table(rows, headers=None):
    if _HAS_TABULATE:
        print(tabulate([list(r) for r in rows], headers=headers, tablefmt='grid'))
    else:
        # Impressão simples
        if headers:
            print(' | '.join(headers))
            print('-' * 40)
        for r in rows:
            print(' | '.join(str(x) for x in r))


def pause():
    input('\nPressione Enter para continuar...')


# ---------- Menus ----------

def menu_main():
    while True:
        clear_screen()
        print('=== Coffee Shops Tia Rosa - Sistema de Gestão ===')
        print('1) Vendas / Registrar pedido')
        print('2) Cardápio / Produtos')
        print('3) Clientes / Fidelidade')
        print('4) Relatórios')
        print('5) Importar/Exportar CSV')
        print('0) Sair')
        opt = input('Escolha uma opção: ').strip()
        if opt == '1':
            menu_sales()
        elif opt == '2':
            menu_products()
        elif opt == '3':
            menu_customers()
        elif opt == '4':
            menu_reports()
        elif opt == '5':
            menu_csv()
        elif opt == '0':
            print('Saindo...')
            break
        else:
            print('Opção inválida.')
            pause()


# ---------- Menu: Produtos ----------

def menu_products():
    while True:
        clear_screen()
        print('--- Gestão de Produtos ---')
        print('1) Listar cardápio (ativo)')
        print('2) Adicionar produto')
        print('3) Editar produto')
        print('4) Remover (inativar) produto')
        print('5) Ver todos os produtos (inclusive inativos)')
        print('0) Voltar')
        opt = input('Opção: ').strip()
        if opt == '1':
            prods = SYSTEM.list_products(only_active=True)
            if not prods:
                print('\nNenhum produto ativo cadastrado.')
            else:
                rows = [(p['id'], p['name'], money(p['price']), p['stock']) for p in prods]
                print_table(rows, headers=['ID', 'Produto', 'Preço', 'Estoque'])
            pause()
        elif opt == '2':
            name = input('Nome: ').strip()
            description = input('Descrição: ').strip()
            price = input('Preço (ex: 7.50): ').strip()
            stock = input('Estoque inicial (int): ').strip()
            try:
                SYSTEM.add_product(name, description, float(price), int(stock))
                print('Produto adicionado com sucesso.')
            except Exception as e:
                print('Erro ao adicionar produto:', e)
            pause()
        elif opt == '3':
            pid = input('ID do produto a editar: ').strip()
            prod = SYSTEM.get_product(pid)
            if not prod:
                print('Produto não encontrado.')
                pause(); continue
            print('Deixe em branco para manter o valor atual.')
            name = input(f'Nome [{prod["name"]}]: ').strip() or prod['name']
            desc = input(f'Descrição [{prod["description"]}]: ').strip() or prod['description']
            price = input(f'Preço [{prod["price"]}]: ').strip() or prod['price']
            stock = input(f'Estoque [{prod["stock"]}]: ').strip() or prod['stock']
            active = input(f'Ativo? (1-sim,0-nao) [{prod["active"]}]: ').strip() or prod['active']
            try:
                SYSTEM.update_product(pid, name=name, description=desc, price=float(price), stock=int(stock), active=int(active))
                print('Produto atualizado.')
            except Exception as e:
                print('Erro:', e)
            pause()
        elif opt == '4':
            pid = input('ID do produto a remover (inativar): ').strip()
            SYSTEM.delete_product(pid)
            print('Produto inativado (soft delete).')
            pause()
        elif opt == '5':
            prods = SYSTEM.list_products(only_active=False)
            rows = [(p['id'], p['name'], money(p['price']), p['stock'], p['active']) for p in prods]
            print_table(rows, headers=['ID', 'Produto', 'Preço', 'Estoque', 'Ativo'])
            pause()
        elif opt == '0':
            break
        else:
            print('Opção inválida.'); pause()


# ---------- Menu: Clientes ----------

def menu_customers():
    while True:
        clear_screen()
        print('--- Clientes e Fidelidade ---')
        print('1) Cadastrar cliente')
        print('2) Listar clientes')
        print('3) Buscar por telefone')
        print('4) Resgatar pontos')
        print('0) Voltar')
        opt = input('Opção: ').strip()
        if opt == '1':
            name = input('Nome: ').strip()
            phone = input('Telefone: ').strip()
            email = input('E-mail: ').strip()
            try:
                SYSTEM.add_customer(name, phone, email)
                print('Cliente cadastrado.')
            except Exception as e:
                print('Erro ao cadastrar:', e)
            pause()
        elif opt == '2':
            rows = SYSTEM.list_customers()
            if not rows:
                print('Nenhum cliente cadastrado.')
            else:
                table = [(r['id'], r['name'], r['phone'], r['email'], r['points']) for r in rows]
                print_table(table, headers=['ID', 'Nome', 'Telefone', 'Email', 'Pontos'])
            pause()
        elif opt == '3':
            phone = input('Telefone para busca: ').strip()
            c = SYSTEM.find_customer_by_phone(phone)
            if not c:
                print('Cliente não encontrado.')
            else:
                print_table([(c['id'], c['name'], c['phone'], c['email'], c['points'])], headers=['ID', 'Nome', 'Telefone', 'Email', 'Pontos'])
            pause()
        elif opt == '4':
            cid = input('ID do cliente: ').strip()
            points = int(input('Quantidade de pontos a resgatar: ').strip())
            ok, msg = SYSTEM.redeem_points(cid, points)
            print(msg)
            pause()
        elif opt == '0':
            break
        else:
            print('Opção inválida.'); pause()


# ---------- Menu: Vendas ----------

def menu_sales():
    clear_screen()
    print('--- Registrar Pedido ---')
    # Seleciona cliente (opcional)
    use_cust = input('Cliente tem cadastro? (s/n): ').strip().lower()
    customer_id = None
    if use_cust == 's':
        phone = input('Informe o telefone do cliente: ').strip()
        cust = SYSTEM.find_customer_by_phone(phone)
        if cust:
            customer_id = cust['id']
            print(f"Cliente: {cust['name']} (Pontos: {cust['points']})")
        else:
            print('Cliente não encontrado. Deseja cadastrar?')
            if input('Cadastrar agora? (s/n): ').strip().lower() == 's':
                name = input('Nome: ').strip()
                email = input('E-mail: ').strip()
                SYSTEM.add_customer(name, phone, email)
                cust = SYSTEM.find_customer_by_phone(phone)
                customer_id = cust['id']
                print('Cliente cadastrado e selecionado.')

    # Monta itens do pedido
    items = []
    while True:
        prods = SYSTEM.list_products(only_active=True)
        if not prods:
            print('Não há produtos disponíveis.'); pause(); return
        print('\n--- Cardápio ---')
        for p in prods:
            print(f"{p['id']}) {p['name']} - {money(p['price'])} - Estoque: {p['stock']}")
        pid = input('\nID do produto (ou 0 para finalizar): ').strip()
        if pid == '0' or pid == '':
            break
        prod = SYSTEM.get_product(pid)
        if not prod:
            print('Produto inválido.'); continue
        qty = input('Quantidade: ').strip()
        try:
            qtyi = int(qty)
            if qtyi <= 0:
                print('Quantidade deve ser maior que zero.'); continue
            if prod['stock'] < qtyi:
                print('Estoque insuficiente.'); continue
            items.append({'product_id': prod['id'], 'qty': qtyi})
        except Exception:
            print('Quantidade inválida.')

    if not items:
        print('Pedido vazio cancelado.'); pause(); return

    payment = input('Forma de pagamento (Dinheiro/Cartao): ').strip() or 'Dinheiro'
    try:
        order_id, total = SYSTEM.create_order(customer_id, items, payment_method=payment)
        print(f'Pedido registrado (ID {order_id}). Total: {money(total)}')
    except Exception as e:
        print('Erro ao criar pedido:', e)
    pause()


# ---------- Menu: Relatórios ----------

def menu_reports():
    while True:
        clear_screen()
        print('--- Relatórios ---')
        print('1) Vendas do dia')
        print('2) Itens mais vendidos')
        print('3) Clientes com mais pontos')
        print('0) Voltar')
        opt = input('Opção: ').strip()
        if opt == '1':
            d = input('Data (YYYY-MM-DD) [hoje]: ').strip() or date.today().isoformat()
            try:
                target = datetime.fromisoformat(d).date()
            except Exception:
                print('Data inválida.'); pause(); continue
            orders, total = SYSTEM.sales_on_date(target)
            print(f'Vendas em {target.isoformat()}: {money(total)}')
            if orders:
                table = [(o['id'], o['customer_id'], o['created_at'], money(o['total']), o['payment_method']) for o in orders]
                print_table(table, headers=['ID', 'ClienteID', 'Criado', 'Total', 'Pagamento'])
            pause()
        elif opt == '2':
            rows = SYSTEM.top_sold_items()
            if not rows:
                print('Sem vendas registradas.')
            else:
                table = [(r['id'], r['name'], r['sold']) for r in rows]
                print_table(table, headers=['ID', 'Produto', 'Qtd Vendida'])
            pause()
        elif opt == '3':
            rows = SYSTEM.customer_rank()
            if not rows:
                print('Nenhum cliente cadastrado.')
            else:
                table = [(r['id'], r['name'], r['points']) for r in rows]
                print_table(table, headers=['ID', 'Nome', 'Pontos'])
            pause()
        elif opt == '0':
            break
        else:
            print('Opção inválida.'); pause()


# ---------- Menu: CSV ----------

def menu_csv():
    while True:
        clear_screen()
        print('--- Importar / Exportar CSV ---')
        print('1) Importar produtos de CSV')
        print('2) Exportar produtos para CSV')
        print('0) Voltar')
        opt = input('Opção: ').strip()
        if opt == '1':
            path = input('Caminho do CSV: ').strip()
            try:
                n = SYSTEM.import_products_csv(path)
                print(f'Importados: {n} produtos (pode haver duplicatas).')
            except Exception as e:
                print('Erro na importação:', e)
            pause()
        elif opt == '2':
            path = input('Caminho para salvar CSV: ').strip()
            try:
                n = SYSTEM.export_products_csv(path)
                print(f'Exportados {n} produtos para {path}.')
            except Exception as e:
                print('Erro na exportação:', e)
            pause()
        elif opt == '0':
            break
        else:
            print('Opção inválida.'); pause()


# -------------------------- Inicialização (seed) --------------------------

def seed_demo_data():
    # Verifica se já existem produtos
    prods = SYSTEM.list_products(only_active=False)
    if prods:
        return
    demo = [
        ('Café Expresso', 'Café passado na máquina - 30ml', 4.50, 50),
        ('Cappuccino', 'Cappuccino com canela', 9.00, 30),
        ('Pão de Queijo', 'Tradicional', 5.00, 40),
        ('Bolo de Cenoura', 'Fatia média', 7.00, 20),
    ]
    for name, desc, price, stock in demo:
        SYSTEM.add_product(name, desc, price, stock)


# -------------------------- Execução --------------------------

if __name__ == '__main__':
    seed_demo_data()
    try:
        menu_main()
    except KeyboardInterrupt:
        print('\nEncerrando...')
    finally:
        SYSTEM.db.conn.close()


=== Coffee Shops Tia Rosa - Sistema de Gestão ===
1) Vendas / Registrar pedido
2) Cardápio / Produtos
3) Clientes / Fidelidade
4) Relatórios
5) Importar/Exportar CSV
0) Sair
Escolha uma opção: 1
--- Registrar Pedido ---
Cliente tem cadastro? (s/n): nao

--- Cardápio ---
4) Bolo de Cenoura - R$ 7,00 - Estoque: 20
1) Café Expresso - R$ 4,50 - Estoque: 50
2) Cappuccino - R$ 9,00 - Estoque: 30
3) Pão de Queijo - R$ 5,00 - Estoque: 40

ID do produto (ou 0 para finalizar): 4
Quantidade: 22
Estoque insuficiente.

--- Cardápio ---
4) Bolo de Cenoura - R$ 7,00 - Estoque: 20
1) Café Expresso - R$ 4,50 - Estoque: 50
2) Cappuccino - R$ 9,00 - Estoque: 30
3) Pão de Queijo - R$ 5,00 - Estoque: 40

ID do produto (ou 0 para finalizar): 4
Quantidade: 20

--- Cardápio ---
4) Bolo de Cenoura - R$ 7,00 - Estoque: 20
1) Café Expresso - R$ 4,50 - Estoque: 50
2) Cappuccino - R$ 9,00 - Estoque: 30
3) Pão de Queijo - R$ 5,00 - Estoque: 40

ID do produto (ou 0 para finalizar): 0
Forma de pagamento (Dinheiro/Car